In [7]:
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer

In [8]:

def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

In [9]:
set_seed(42)

In [10]:
model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_paraphraser')
# model = pickle.load(open("pretrained_t5", 'rb'))
tokenizer = T5Tokenizer.from_pretrained('t5-base')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print ("device ",device)
model = model.to(device)

In [29]:
def paraphrase(sentence, num_similar=10):
    # sentence = "If f(theta) is 3 times theta plus 19 squared and theta is 1 what is f(theta)?"
    text =  "paraphrase: " + sentence + " </s>"
    encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

    # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
    beam_outputs = model.generate(
        input_ids=input_ids, attention_mask=attention_masks,
        do_sample=True,
        max_length=1024,
        top_k=120,
        top_p=1,
        early_stopping=True,
        num_return_sequences=15
    )


    # print ("\nOriginal Question ::")
    # print (sentence)
    # print ("\n")
    # print ("Paraphrased Questions :: ")
    final_outputs =[]
    for beam_output in beam_outputs:
        sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
        if sent.lower() != sentence.lower() and sent not in final_outputs:
            final_outputs.append(sent)

    # for i, final_output in enumerate(final_outputs):
    #     print("{}: {}".format(i, final_output))
    return final_outputs[:num_similar]

In [30]:
result = paraphrase("Write a procedure that takes a 2D array and returns the final column as a two dimensional array.", 10)
print(len(result))
print(result)

10
['Write a procedure that takes a 2D array and returns the last column as a two dimensional array.', 'Write a procedure that gives a 2D array and returns the final column as a 2D array.', 'Write a procedure that takes a 2D array and returns the last column as a two dimensional array. *Note: This procedure takes 2D arrays into account.', 'Write a procedure that takes a base 2D array and returns the final columns as a two dimensional array.', 'Write a procedure that takes a 2D array and returns the final column as two dimensional array.', 'Write a procedure that takes a 2D array and returns the final column as a two dimensional array, but the object is a number. In each case, the procedure should take two dediments. Return dediments.In each case, if the array is 2D, may be different.', 'Write a procedure that takes a 2D array and return the final column as a 2D array.', 'Write a procedure that takes a 2D array if it returns the final column as a two dimensional array.', 'Write a proced